In [4]:
import sys
import os
# Add parent directory to Python path so it can find security_env.py
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

import numpy as np
import optuna
from optuna.samplers import TPESampler
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from security_env import SecurityEnv

def objective(trial):
    """
    Objective function for Optuna optimization
    """
    # Define hyperparameter ranges
    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-3),
        'n_steps': trial.suggest_categorical('n_steps', [2048, 4096, 8192]),
        'batch_size': trial.suggest_categorical('batch_size', [32, 64, 128]),
        'n_epochs': trial.suggest_categorical('n_epochs', [5, 10, 20]),
        'gamma': trial.suggest_categorical('gamma', [0.95, 0.99, 0.995]),
        'gae_lambda': trial.suggest_categorical('gae_lambda', [0.9, 0.95, 0.98]),
        'clip_range': trial.suggest_categorical('clip_range', [0.1, 0.2, 0.3]),
        'ent_coef': trial.suggest_loguniform('ent_coef', 1e-5, 1e-2),
        'alpha': trial.suggest_categorical('alpha', [0.3, 0.5, 0.7]),
        'beta': trial.suggest_categorical('beta', [0.3, 0.5, 0.7]),
        's_min': trial.suggest_categorical('s_min', [5.0, 8.0, 10.0])
    }
    
    # Create environment
    env = SecurityEnv(
        alpha=params['alpha'],
        beta=params['beta'],
        s_min=params['s_min']
    )
    env = DummyVecEnv([lambda: env])
    
    # Create and train model
    model = PPO(
        "MlpPolicy",
        env,
        learning_rate=params['learning_rate'],
        n_steps=params['n_steps'],
        batch_size=params['batch_size'],
        n_epochs=params['n_epochs'],
        gamma=params['gamma'],
        gae_lambda=params['gae_lambda'],
        clip_range=params['clip_range'],
        ent_coef=params['ent_coef'],
        verbose=0
    )
    
    # Train model
    model.learn(total_timesteps=10000)
    
    # Evaluate model
    eval_env = SecurityEnv(
        alpha=params['alpha'],
        beta=params['beta'],
        s_min=params['s_min']
    )
    eval_env = DummyVecEnv([lambda: eval_env])
    
    mean_reward, _ = evaluate_model(model, eval_env)
    
    return mean_reward

def run_bayesian_optimization(n_trials=50):
    """
    Run Bayesian Optimization using Optuna
    """
    # Create study
    study = optuna.create_study(
        direction='maximize',
        sampler=TPESampler(seed=42)
    )
    
    # Run optimization
    study.optimize(objective, n_trials=n_trials)
    
    # Print best results
    print("Best trial:")
    trial = study.best_trial
    print("  Value: ", trial.value)
    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")
    
    # Save results
    results_df = study.trials_dataframe()
    results_df.to_csv('bayesian_optimization_results.csv', index=False)
    
    return study

def evaluate_model(model, env, n_episodes=5):
    """Evaluate a trained model"""
    rewards = []
    for _ in range(n_episodes):
        obs = env.reset()
        done = False
        episode_reward = 0
        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done, info = env.step(action)
            episode_reward += reward
        rewards.append(episode_reward)
    return np.mean(rewards), np.std(rewards)

In [5]:
study = run_bayesian_optimization(n_trials=50)

[I 2025-04-29 22:38:49,389] A new study created in memory with name: no-name-e86f932e-c773-48b1-876a-86b8165c5671
C:\conda_tmp\ipykernel_20524\244527143.py:21: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-3),
C:\conda_tmp\ipykernel_20524\244527143.py:28: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'ent_coef': trial.suggest_loguniform('ent_coef', 1e-5, 1e-2),


Feature names: ['Familarity', 'Frequency of Password Changes', 'Difficulty Level Password', 'Effort Required Password', 'Perceived Importance Password', 'Password Uniqueness', 'Frequency of MFA prompts', 'Difficulty Level MFA', 'Effort Required MFA', 'Perceived Importance of MFA', 'Frequency of Security Warnings', 'Difficulty Level Security Warnings', 'Effort Required Security Warnings', 'Perceived Importance of Security Warnings', 'Warnings Response Behaviour', 'Hardware security key (FIDO2 token) or cryptographic device', 'On-device prompt or biometric', 'OTP via authenticator app', 'OTP via SMS/email', 'Secondary email/phone or security questions', 'No MFA enabled']
Feature names: ['Familarity', 'Frequency of Password Changes', 'Difficulty Level Password', 'Effort Required Password', 'Perceived Importance Password', 'Password Uniqueness', 'Frequency of MFA prompts', 'Difficulty Level MFA', 'Effort Required MFA', 'Perceived Importance of MFA', 'Frequency of Security Warnings', 'Diffi

[I 2025-04-29 22:42:15,833] Trial 0 finished with value: 2512.673828125 and parameters: {'learning_rate': 5.6115164153345e-05, 'n_steps': 2048, 'batch_size': 32, 'n_epochs': 5, 'gamma': 0.99, 'gae_lambda': 0.9, 'clip_range': 0.2, 'ent_coef': 7.476312062252303e-05, 'alpha': 0.3, 'beta': 0.7, 's_min': 10.0}. Best is trial 0 with value: 2512.673828125.


Feature names: ['Familarity', 'Frequency of Password Changes', 'Difficulty Level Password', 'Effort Required Password', 'Perceived Importance Password', 'Password Uniqueness', 'Frequency of MFA prompts', 'Difficulty Level MFA', 'Effort Required MFA', 'Perceived Importance of MFA', 'Frequency of Security Warnings', 'Difficulty Level Security Warnings', 'Effort Required Security Warnings', 'Perceived Importance of Security Warnings', 'Warnings Response Behaviour', 'Hardware security key (FIDO2 token) or cryptographic device', 'On-device prompt or biometric', 'OTP via authenticator app', 'OTP via SMS/email', 'Secondary email/phone or security questions', 'No MFA enabled']
Feature names: ['Familarity', 'Frequency of Password Changes', 'Difficulty Level Password', 'Effort Required Password', 'Perceived Importance Password', 'Password Uniqueness', 'Frequency of MFA prompts', 'Difficulty Level MFA', 'Effort Required MFA', 'Perceived Importance of MFA', 'Frequency of Security Warnings', 'Diffi

[I 2025-04-29 22:46:00,342] Trial 1 finished with value: 3264.388671875 and parameters: {'learning_rate': 1.2385137298860926e-05, 'n_steps': 2048, 'batch_size': 64, 'n_epochs': 20, 'gamma': 0.995, 'gae_lambda': 0.95, 'clip_range': 0.1, 'ent_coef': 0.00043664735929796326, 'alpha': 0.5, 'beta': 0.3, 's_min': 5.0}. Best is trial 1 with value: 3264.388671875.


...

Feature names: ['Familarity', 'Frequency of Password Changes', 'Difficulty Level Password', 'Effort Required Password', 'Perceived Importance Password', 'Password Uniqueness', 'Frequency of MFA prompts', 'Difficulty Level MFA', 'Effort Required MFA', 'Perceived Importance of MFA', 'Frequency of Security Warnings', 'Difficulty Level Security Warnings', 'Effort Required Security Warnings', 'Perceived Importance of Security Warnings', 'Warnings Response Behaviour', 'Hardware security key (FIDO2 token) or cryptographic device', 'On-device prompt or biometric', 'OTP via authenticator app', 'OTP via SMS/email', 'Secondary email/phone or security questions', 'No MFA enabled']
Feature names: ['Familarity', 'Frequency of Password Changes', 'Difficulty Level Password', 'Effort Required Password', 'Perceived Importance Password', 'Password Uniqueness', 'Frequency of MFA prompts', 'Difficulty Level MFA', 'Effort Required MFA', 'Perceived Importance of MFA', 'Frequency of Security Warnings', 'Diffi

[I 2025-04-30 01:14:32,665] Trial 48 finished with value: 5917.8251953125 and parameters: {'learning_rate': 3.417449984662684e-05, 'n_steps': 4096, 'batch_size': 128, 'n_epochs': 10, 'gamma': 0.995, 'gae_lambda': 0.9, 'clip_range': 0.3, 'ent_coef': 0.006793773622155831, 'alpha': 0.7, 'beta': 0.5, 's_min': 10.0}. Best is trial 10 with value: 6611.9716796875.


Feature names: ['Familarity', 'Frequency of Password Changes', 'Difficulty Level Password', 'Effort Required Password', 'Perceived Importance Password', 'Password Uniqueness', 'Frequency of MFA prompts', 'Difficulty Level MFA', 'Effort Required MFA', 'Perceived Importance of MFA', 'Frequency of Security Warnings', 'Difficulty Level Security Warnings', 'Effort Required Security Warnings', 'Perceived Importance of Security Warnings', 'Warnings Response Behaviour', 'Hardware security key (FIDO2 token) or cryptographic device', 'On-device prompt or biometric', 'OTP via authenticator app', 'OTP via SMS/email', 'Secondary email/phone or security questions', 'No MFA enabled']
Feature names: ['Familarity', 'Frequency of Password Changes', 'Difficulty Level Password', 'Effort Required Password', 'Perceived Importance Password', 'Password Uniqueness', 'Frequency of MFA prompts', 'Difficulty Level MFA', 'Effort Required MFA', 'Perceived Importance of MFA', 'Frequency of Security Warnings', 'Diffi

[I 2025-04-30 01:17:58,542] Trial 49 finished with value: 2684.553955078125 and parameters: {'learning_rate': 7.855459970676167e-05, 'n_steps': 4096, 'batch_size': 32, 'n_epochs': 10, 'gamma': 0.95, 'gae_lambda': 0.9, 'clip_range': 0.2, 'ent_coef': 0.005092477796407556, 'alpha': 0.3, 'beta': 0.5, 's_min': 8.0}. Best is trial 10 with value: 6611.9716796875.


Best trial:
  Value:  6611.9716796875
  Params: 
    learning_rate: 4.194601131160366e-05
    n_steps: 4096
    batch_size: 128
    n_epochs: 10
    gamma: 0.995
    gae_lambda: 0.9
    clip_range: 0.3
    ent_coef: 0.0011746931059660698
    alpha: 0.7
    beta: 0.5
    s_min: 5.0
